In [5]:
import numpy as np
from __future__ import division
import sqlite3

In [6]:
sqlitefile = 'data.sqlite'

# Create database

In [7]:
conn = sqlite3.connect(sqlitefile)
c = conn.cursor()
c.execute('''CREATE TABLE sanders
       (datetime       REAL,
       sentiment      REAL,
       tweetID        TEXT);''')
c.execute('''CREATE TABLE trump
       (datetime       REAL,
       sentiment      REAL,
       tweetID        TEXT);''')
c.execute('''CREATE TABLE clinton
       (datetime       REAL,
       sentiment      REAL,
       tweetID        TEXT);''')
c.execute('''CREATE TABLE cruz
       (datetime       REAL,
       sentiment      REAL,
       tweetID        TEXT);''')
c.execute('''CREATE TABLE unknown
       (datetime       REAL,
       sentiment      REAL,
       tweetID        TEXT);''')
conn.commit()
conn.close()

# Insert data

In [8]:
import pandas as pd

In [ ]:
candidates = ['sanders', 'trump', 'clinton', 'cruz', 'unknown']
for candidate in candidates:
    data = pd.read_csv('data_'+candidate+'_00000.csv', header=None,
                   names=['date', 'sentiment', 'tweetID'],
                   dtype={'date': np.float64, 'sentiment': np.float64, 'tweetID': str})
    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()
    c.executemany('''INSERT INTO '''+candidate+'''(datetime, sentiment, tweetID) VALUES (?, ?, ?)''', np.array(data))
    conn.commit()
    conn.close()

# Create downsampled database

In [ ]:
sqlitefile_downsampled = 'data_downsampled.sqlite'

In [ ]:
conn = sqlite3.connect(sqlitefile_downsampled)
c = conn.cursor()
c.execute('''CREATE TABLE sanders
       (datetime      REAL,
       sentiment      REAL,
       tweet_count    INTEGER);''')
c.execute('''CREATE TABLE trump
       (datetime      REAL,
       sentiment      REAL,
       tweet_count    INTEGER);''')
c.execute('''CREATE TABLE clinton
       (datetime      REAL,
       sentiment      REAL,
       tweet_count    INTEGER);''')
c.execute('''CREATE TABLE cruz
       (datetime      REAL,
       sentiment      REAL,
       tweet_count    INTEGER);''')
conn.commit()
conn.close()

<hr>
# Downsample

In [48]:
bin_size = 30 * 60 # 30 minutes, in seconds
min_tweets = 100 # minimum number of tweets for a valid entry

candidates = ['sanders', 'trump', 'clinton', 'cruz']

for candidate in candidates:
    
    conn = sqlite3.connect(sqlitefile_downsampled)
    c = conn.cursor()
#     c.execute('''DELETE FROM '''+candidate+''' WHERE 1=1''')
#     conn.commit()
    c.execute('''SELECT * FROM '''+candidate+''' LIMIT 1;''')
    row = c.fetchone()
    conn.close()
    
    if not row:
        # table is empty, start from scratch

        conn = sqlite3.connect(sqlitefile)
        c = conn.cursor()
        c.execute('''SELECT * FROM '''+candidate+''';''')
        all_rows = c.fetchall()
        conn.close()
        
        prev_time = None
        sentiments = []
        tweet_count = 0
        for row in all_rows:
            sentiments.append(row[1])
            tweet_count += 1
            if not prev_time:
                prev_time = row[0]
                continue
            time = row[0]
            if time - prev_time > bin_size:
                # we've passed bin_size, wrap it up
                if time - prev_time > bin_size*2:
                    # more than 2 bin_sizes have passed, we're missing data. Add an empty entry.
                    conn = sqlite3.connect(sqlitefile_downsampled)
                    c = conn.cursor()
                    c.execute('''INSERT INTO '''+candidate+'''(datetime, sentiment, tweet_count) VALUES (?,?,?);''',
                             (time - (time-prev_time)/2, None, tweet_count))
                    conn.commit()
                    conn.close()
                    prev_time = time
                    sentiments = []
                    tweet_count = 0
                elif tweet_count >= min_tweets: # check if we have a reasonable number of tweets to get a mean sentiment from
                    conn = sqlite3.connect(sqlitefile_downsampled)
                    c = conn.cursor()
                    c.execute('''INSERT INTO '''+candidate+'''(datetime, sentiment, tweet_count) VALUES (?,?,?);''',
                             (time - (time-prev_time)/2, np.mean(sentiments), tweet_count))
                    conn.commit()
                    conn.close()
                    prev_time = time
                    sentiments = []
                    tweet_count = 0
        
    else: # table is not empty
        
        conn = sqlite3.connect(sqlitefile_downsampled)
        c = conn.cursor()
        c.execute('''SELECT datetime FROM '''+candidate+''' ORDER BY rowid DESC LIMIT 1;''')
        row = c.fetchone()
        downsampled_lasttime = row[0]
        conn.close()
        
        conn = sqlite3.connect(sqlitefile)
        c = conn.cursor()
        c.execute('''SELECT datetime FROM '''+candidate+''' ORDER BY rowid DESC LIMIT 1;''')
        row = c.fetchone()
        data_lasttime = row[0]
        conn.close()
        
        if data_lasttime - downsampled_lasttime > bin_size:
            
            # time to add another datapoint

            conn = sqlite3.connect(sqlitefile)
            c = conn.cursor()
            c.execute('''SELECT * FROM '''+candidate+''' WHERE datetime > '''+str(downsampled_lasttime)+''';''')
            rows = c.fetchall()
            conn.close()

            prev_time = downsampled_lasttime
            sentiments = []
            tweet_count = 0
            for row in rows:
                sentiments.append(row[1])
                tweet_count += 1
                if not prev_time:
                    prev_time = row[0]
                    continue
                time = row[0]
                if time - prev_time > bin_size:
                    # we've passed bin_size, wrap it up
                    if time - prev_time > bin_size*2:
                        # more than 2 bin_sizes have passed, we're missing data. Add an empty entry.
                        conn = sqlite3.connect(sqlitefile_downsampled)
                        c = conn.cursor()
                        c.execute('''INSERT INTO '''+candidate+'''(datetime, sentiment, tweet_count) VALUES (?,?,?);''',
                                 (time - (time-prev_time)/2, None, tweet_count))
                        conn.commit()
                        conn.close()
                    elif tweet_count >= min_tweets: # check if we have a reasonable number of tweets to get a mean sentiment from
                        conn = sqlite3.connect(sqlitefile_downsampled)
                        c = conn.cursor()
                        c.execute('''INSERT INTO '''+candidate+'''(datetime, sentiment, tweet_count) VALUES (?,?,?);''',
                                 (time - (time-prev_time)/2, np.mean(sentiments), tweet_count))
                        conn.commit()
                        conn.close()
                    
                    prev_time = time
                    sentiments = []
                    tweet_count = 0